In [103]:
import snappy
import ast
import csv
import pickle

Lets import all pd codes of the census knots

In [104]:
pd_codes=[]
with open('/Users/leomousseau/Desktop/Bachelorarbeit/pd_codes_census','r') as inp:
    reader=csv.reader(inp)
    for row in reader:
        pd_codes.append(row)

for k in pd_codes:
    k[1]=eval(k[1])

pd_dict=dict(pd_codes)

lets first find lower bounds on the slice genus. We only need to consider the non SQP ones, since we have exact values for the others

lets import all known tau-invariant values of the census knots

In [105]:
tau = []
with open('/Users/leomousseau/Desktop/Bachelor code census/knot floer data.csv', 'r') as inp:
    reader = csv.reader(inp)
    for row in reader:
        try:
            tau.append([row[0], float(row[7])])
        except ValueError:
            continue
del tau[0]
tau_dict=dict(tau)


In [106]:
lower_bounds_slice=[]
for k in pd_codes:
    try:
        tau=abs(tau_dict[k[0]])
    except:
        tau=0
    lower_bounds_slice.append([k[0],tau])
  
    

Lets now compute lower bounds on the topological genus of census knots via the bound (1/2)*abs(signature)=< g^top

In [107]:
lower_bounds_top=[]
for k in pd_codes:
    s=snappy.Link(k[1]).signature()
    b=(1/2)*abs(s)
    lower_bounds_top.append([k[0],b])

Lets use the SnapPy built-in function "slice_obstruction_HKL(primes_spec, verbose=False, check_in_S3=True)":

For the exterior of a knot in S^3, this function searches for a topological slicing obstruction from:

Herald, Kirk, Livingston, Math Zeit., 2010 https://dx.doi.org/10.1007/s00209-009-0548-1 https://arxiv.org/abs/0804.1355

In [ ]:
spec = [(10, [0, 20]), (20, [0, 10])]
for k in lower_bounds_top:
    L=snappy.Link(pd_dict[k[0]])
    M=L.exterior()
    if k[1]==0: # lets try to obstruct the topological sliceness via the
        try:
            if type(M.slice_obstruction_HKL(spec, verbose=False))==type((1,2)):
                k[1]=1 #since the topological sliceness is obstructed
                print('worked')
        except:
            print('something wrong with HKL')


In [109]:
lower_bounds_top_dict=dict(lower_bounds_top) 

Lets use the obvious inequalities of the slice genera to update the lower slice genus bounds:

In [ ]:
for k in lower_bounds_slice:
    top_bound=lower_bounds_top_dict[k[0]]
    if k[1]<top_bound:
        k[1]=top_bound

In [124]:
lower_bounds_slice_dict=dict(lower_bounds_slice)

Let's now combine the lower bounds with the upper bounds to get the final bounds of all census knots:

In [126]:
upper_bounds=[]
with open('/Users/leomousseau/Desktop/Bachelor code census/upper_bounds_census_final.csv','r') as inp:
    reader=csv.reader(inp)
    for row in reader:
        upper_bounds.append(row)
for k in upper_bounds:
    k[1]=eval(k[1])

In [138]:
final_bounds=[]
for k in upper_bounds:
    lower_slice=lower_bounds_slice_dict[k[0]]
    if lower_slice==int(lower_slice):
        lower_slice=int(lower_slice)
    
    lower_top=lower_bounds_top_dict[k[0]]
    upper_slice=k[1][0]
    upper_top=k[1][1]
    if lower_slice>upper_slice:
        print('error 1',k)
    if lower_top>upper_top:
        print('error 2',k)
    if lower_top>upper_slice:
        print('error 3')
    
    final_bounds.append([k[0],(lower_slice,upper_slice),(lower_top,upper_top)])
for k in final_bounds:
    t1=k[1]
    t2=k[2]
    if t1[0]==t1[1]:
        k[1]=t1[0]
    if t2[0]==t2[1]:
        k[2]=t2[0]

In [146]:
with open('final_slice_genera_bounds_census_knots.csv','w') as out:
    writer=csv.writer(out)
    for row in final_bounds:
        writer.writerow(row)